In [3]:
# ====================================================
# 🚀 ULTRA-OPTIMIZED Real-ESRGAN | High-Res + Realistic
# Features: Tiling, FP16, Multiple Models, Face Enhancement
# ====================================================

# 1. 📦 Install optimized dependencies
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 --quiet
!pip install realesrgan basicsr gfpgan facexlib opencv-python-headless Pillow numpy --quiet

# 2. 🩹 Fix PyTorch 2.x compatibility
!sed -i 's#from torchvision.transforms.functional_tensor import rgb_to_grayscale#from torchvision.transforms.functional import rgb_to_grayscale#g' /usr/local/lib/python3.*/dist-packages/basicsr/data/degradations.py

# 3. 📥 Download ALL model weights for best quality
import os
import urllib.request
from pathlib import Path

def download_model(url, filename):
    if not os.path.exists(filename):
        print(f"📥 Downloading {filename}...")
        urllib.request.urlretrieve(url, filename)
        print(f"✅ {filename} ready!")
    else:
        print(f"✅ {filename} already exists")

# Real-ESRGAN x4 (General photos - Best quality)
download_model(
    'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth',
    'RealESRGAN_x4plus.pth'
)

# Real-ESRGAN x2 (For less aggressive upscaling)
download_model(
    'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth',
    'RealESRGAN_x2plus.pth'
)

# RealESRNet x4 (Sharper, more detailed)
download_model(
    'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.1/RealESRNet_x4plus.pth',
    'RealESRNet_x4plus.pth'
)

# GFPGAN for face enhancement
download_model(
    'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth',
    'GFPGANv1.3.pth'
)

print("\n🎉 All models downloaded!\n")

# ====================================================
# 🧠 Optimized Upscaler Class with Advanced Features
# ====================================================
import torch
import numpy as np
from PIL import Image
import cv2
from realesrgan import RealESRGANer
from basicsr.archs.rrdbnet_arch import RRDBNet
from gfpgan import GFPGANer

class UltraUpscaler:
    """
    Ultra-optimized image super-resolution with:
    - Automatic tiling for huge images
    - FP16 for 2x faster inference
    - Multiple model support
    - Face enhancement
    - Memory-efficient processing
    """

    def __init__(
        self,
        model_name='RealESRGAN_x4plus',
        tile_size=512,
        tile_pad=10,
        pre_pad=0,
        use_fp16=True,
        face_enhance=False
    ):
        """
        Initialize upscaler

        Args:
            model_name: 'RealESRGAN_x4plus', 'RealESRGAN_x2plus', 'RealESRNet_x4plus'
            tile_size: Tile size (larger = faster but more VRAM, 0 = no tiling)
            tile_pad: Overlap between tiles to avoid seams
            pre_pad: Pre-padding size
            use_fp16: Use half precision (2x faster on modern GPUs)
            face_enhance: Enable GFPGAN face enhancement
        """
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.use_fp16 = use_fp16 and self.device == 'cuda'
        self.face_enhance = face_enhance

        print(f"🔧 Initializing {model_name} on {self.device.upper()}")
        print(f"⚡ FP16: {self.use_fp16} | Tile Size: {tile_size if tile_size > 0 else 'Disabled'}")

        # Model configurations
        model_configs = {
            'RealESRGAN_x4plus': {
                'scale': 4,
                'num_block': 23,
                'model_path': 'RealESRGAN_x4plus.pth'
            },
            'RealESRGAN_x2plus': {
                'scale': 2,
                'num_block': 23,
                'model_path': 'RealESRGAN_x2plus.pth'
            },
            'RealESRNet_x4plus': {
                'scale': 4,
                'num_block': 23,
                'model_path': 'RealESRNet_x4plus.pth'
            }
        }

        config = model_configs[model_name]

        # Initialize model architecture
        model_arch = RRDBNet(
            num_in_ch=3,
            num_out_ch=3,
            num_feat=64,
            num_block=config['num_block'],
            num_grow_ch=32,
            scale=config['scale']
        )

        # Create upsampler
        self.upsampler = RealESRGANer(
            scale=config['scale'],
            model_path=config['model_path'],
            model=model_arch,
            tile=tile_size,
            tile_pad=tile_pad,
            pre_pad=pre_pad,
            half=self.use_fp16,
            device=self.device
        )

        # Initialize face enhancer if requested
        self.face_enhancer = None
        if face_enhance:
            print("👤 Loading GFPGAN for face enhancement...")
            self.face_enhancer = GFPGANer(
                model_path='GFPGANv1.3.pth',
                upscale=config['scale'],
                arch='clean',
                channel_multiplier=2,
                bg_upsampler=self.upsampler,
                device=self.device
            )
            print("✅ Face enhancer ready!")

        print("✅ Model loaded successfully!\n")

    def enhance(
        self,
        input_path,
        output_path=None,
        outscale=None,
        output_format='png',
        quality=95,
        denoise_strength=0.5
    ):
        """
        Enhance single image with optimizations

        Args:
            input_path: Input image path
            output_path: Output path (auto-generated if None)
            outscale: Final scale (None = use model scale)
            output_format: 'png', 'jpg', 'webp'
            quality: Output quality (0-100)
            denoise_strength: Denoise strength (0.0-1.0)

        Returns:
            Path to enhanced image
        """
        print(f"📸 Processing: {input_path}")

        # Load image efficiently
        img = cv2.imread(input_path, cv2.IMREAD_COLOR)
        if img is None:
            raise ValueError(f"Cannot read image: {input_path}")

        h, w = img.shape[:2]
        print(f"📏 Original: {w}x{h} pixels ({w*h/1e6:.1f}MP)")

        # Memory check for large images
        estimated_vram = (w * h * 3 * 4 * 16) / (1024**3)  # GB
        print(f"💾 Estimated VRAM usage: ~{estimated_vram:.2f}GB")

        # Enhance with appropriate method
        try:
            if self.face_enhance and self.face_enhancer:
                print("👤 Enhancing with face restoration...")
                _, _, output = self.face_enhancer.enhance(
                    img,
                    has_aligned=False,
                    only_center_face=False,
                    paste_back=True,
                    weight=denoise_strength
                )
            else:
                print("🎨 Enhancing with Real-ESRGAN...")
                output, _ = self.upsampler.enhance(
                    img,
                    outscale=outscale
                )
        except RuntimeError as e:
            if "out of memory" in str(e):
                print("⚠️  GPU OOM! Try reducing tile_size or use CPU")
                # Clear cache and retry with smaller tiles
                torch.cuda.empty_cache()
                self.upsampler.tile = 256
                output, _ = self.upsampler.enhance(img, outscale=outscale)
            else:
                raise e

        # Post-processing for maximum realism
        output = self._enhance_realism(output)

        # Generate output path
        if output_path is None:
            stem = Path(input_path).stem
            output_path = f"{stem}_ultra_{output.shape[1]}x{output.shape[0]}.{output_format}"

        # Save with optimal settings
        self._save_image(output, output_path, output_format, quality)

        new_h, new_w = output.shape[:2]
        print(f"✨ Enhanced: {new_w}x{new_h} pixels ({new_w*new_h/1e6:.1f}MP)")
        print(f"💾 Saved: {output_path}\n")

        return output_path

    def _enhance_realism(self, img):
        """Apply subtle enhancements for photo-realism"""
        # Preserve original if already high quality
        if img.dtype != np.uint8:
            img = np.clip(img, 0, 255).astype(np.uint8)

        # Very subtle sharpening (enhances fine details)
        kernel = np.array([
            [0, -0.25, 0],
            [-0.25, 2, -0.25],
            [0, -0.25, 0]
        ])
        sharpened = cv2.filter2D(img, -1, kernel)
        img = cv2.addWeighted(img, 0.7, sharpened, 0.3, 0)

        # Slight color vibrance boost (improves realism)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV).astype(np.float32)
        hsv[:, :, 1] = hsv[:, :, 1] * 1.05  # +5% saturation
        hsv[:, :, 1] = np.clip(hsv[:, :, 1], 0, 255)
        img = cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR)

        return img

    def _save_image(self, img, path, fmt, quality):
        """Save image with optimal compression"""
        fmt = fmt.lower()

        if fmt in ['jpg', 'jpeg']:
            # JPEG with optimal quality
            cv2.imwrite(path, img, [
                cv2.IMWRITE_JPEG_QUALITY, quality,
                cv2.IMWRITE_JPEG_OPTIMIZE, 1,
                cv2.IMWRITE_JPEG_PROGRESSIVE, 1
            ])
        elif fmt == 'png':
            # PNG with compression
            compression = int((100 - quality) / 10)
            cv2.imwrite(path, img, [
                cv2.IMWRITE_PNG_COMPRESSION, min(compression, 9)
            ])
        elif fmt == 'webp':
            # WebP for best compression/quality ratio
            cv2.imwrite(path, img, [
                cv2.IMWRITE_WEBP_QUALITY, quality
            ])
        else:
            cv2.imwrite(path, img)

    def batch_enhance(
        self,
        input_folder,
        output_folder='upscaled_output',
        **kwargs
    ):
        """
        Batch process folder with progress tracking

        Args:
            input_folder: Input folder path
            output_folder: Output folder path
            **kwargs: Additional arguments for enhance()
        """
        os.makedirs(output_folder, exist_ok=True)

        # Find all images
        extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.webp', '.tiff'}
        input_path = Path(input_folder)
        images = [f for f in input_path.iterdir() if f.suffix.lower() in extensions]

        total = len(images)
        print(f"📁 Found {total} images to process\n")

        for i, img_path in enumerate(images, 1):
            print(f"[{i}/{total}] ", end='')
            try:
                output_path = os.path.join(
                    output_folder,
                    f"{img_path.stem}_enhanced{img_path.suffix}"
                )
                self.enhance(str(img_path), output_path, **kwargs)
            except Exception as e:
                print(f"❌ Error: {e}\n")
                continue

        print(f"🎉 Batch processing complete! Check '{output_folder}' folder")


# ====================================================
# 🎯 Usage Examples - Choose Your Configuration
# ====================================================

# Configuration presets:

# 1️⃣ MAXIMUM QUALITY (Slow, best for portraits/products)
upscaler_max = UltraUpscaler(
    model_name='RealESRGAN_x4plus',
    tile_size=0,  # No tiling = best quality
    use_fp16=True,
    face_enhance=True  # Enable for portraits
)

# 2️⃣ BALANCED (Fast + Good quality, recommended)
upscaler_balanced = UltraUpscaler(
    model_name='RealESRGAN_x4plus',
    tile_size=512,
    use_fp16=True,
    face_enhance=False
)

# 3️⃣ ULTRA-HIGH-RES (For huge images > 10MP)
upscaler_huge = UltraUpscaler(
    model_name='RealESRGAN_x4plus',
    tile_size=256,  # Small tiles for memory efficiency
    tile_pad=10,
    use_fp16=True,
    face_enhance=False
)

# 4️⃣ SUBTLE 2x (Less aggressive, natural look)
upscaler_2x = UltraUpscaler(
    model_name='RealESRGAN_x2plus',
    tile_size=512,
    use_fp16=True
)

# ====================================================
# 📤 Example Usage
# ====================================================

# Single image enhancement:
# upscaler_balanced.enhance(
#     'your_image.jpg',
#     output_path='enhanced.png',
#     output_format='png',
#     quality=95
# )

# Batch processing:
# upscaler_balanced.batch_enhance(
#     input_folder='input_images',
#     output_folder='upscaled_results',
#     output_format='png',
#     quality=95
# )

# For portraits with face enhancement:
# upscaler_max.enhance(
#     'portrait.jpg',
#     output_format='png',
#     quality=98,
#     denoise_strength=0.7  # Higher = smoother faces
# )

print("🚀 Ready to enhance! Use one of the upscaler instances above.")
print("💡 Tip: Start with upscaler_balanced for best speed/quality trade-off")

✅ RealESRGAN_x4plus.pth already exists
✅ RealESRGAN_x2plus.pth already exists
✅ RealESRNet_x4plus.pth already exists
✅ GFPGANv1.3.pth already exists

🎉 All models downloaded!

🔧 Initializing RealESRGAN_x4plus on CUDA
⚡ FP16: True | Tile Size: Disabled
👤 Loading GFPGAN for face enhancement...


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth" to /content/gfpgan/weights/detection_Resnet50_Final.pth



100%|██████████| 104M/104M [00:00<00:00, 221MB/s] 


Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.2.2/parsing_parsenet.pth" to /content/gfpgan/weights/parsing_parsenet.pth



100%|██████████| 81.4M/81.4M [00:00<00:00, 200MB/s]


✅ Face enhancer ready!
✅ Model loaded successfully!

🔧 Initializing RealESRGAN_x4plus on CUDA
⚡ FP16: True | Tile Size: 512
✅ Model loaded successfully!

🔧 Initializing RealESRGAN_x4plus on CUDA
⚡ FP16: True | Tile Size: 256
✅ Model loaded successfully!

🔧 Initializing RealESRGAN_x2plus on CUDA
⚡ FP16: True | Tile Size: 512
✅ Model loaded successfully!

🚀 Ready to enhance! Use one of the upscaler instances above.
💡 Tip: Start with upscaler_balanced for best speed/quality trade-off


In [8]:
upscaler_max.enhance('portrait.jpg')

📸 Processing: portrait.jpg
📏 Original: 1080x1433 pixels (1.5MP)
💾 Estimated VRAM usage: ~0.28GB
👤 Enhancing with face restoration...
✨ Enhanced: 4320x5732 pixels (24.8MP)
💾 Saved: portrait_ultra_4320x5732.png



'portrait_ultra_4320x5732.png'

In [7]:
# # For portraits with faces (BEST QUALITY)
# upscaler_max.enhance('portrait.jpg', face_enhance=True)

# # For general photos (BALANCED)
# upscaler_balanced.enhance('photo.jpg')

# # For huge images >10MP (MEMORY EFFICIENT)
# upscaler_huge.enhance('huge_image.jpg')

# # For subtle enhancement (NATURAL 2x)
# upscaler_2x.enhance('image.jpg')